In [6]:
import numpy as np 
human, bot = 1, -1 

Next cell implements `State`. `State` implements many utility functions which will then be used in our implementation of the minimax algorithm. Any problem implmenting `next_actions`, `next_state` and `is_terminal` can be fed into minimax. 

In [7]:
class State :
    def __init__(self, board=None) : 
        if board is not None : 
            self.board = board 
        else : 
            self.board = np.zeros((3,3))

    def next_actions(self) : 
        """
        Returns a generator yielding empty 
        spots on the board.
        """
        for x in range(3) : 
            for y in range(3) :
                if self.board[x][y] == 0 : 
                    yield x, y 

    def next_state(self, x, y, player) : 
        """
        Setting empty cell (x, y) on the board
        to player's move, which can be +1 or -1.
        """
        board = self.board.copy() 
        board[x][y] = player  
        return State(board)
    
    def move(self, x, y, player) : 
        self.board[x][y] = player 
    
    def print_board(self) : 
        """
        Utility function to print the board
        representing current state.
        """
        for i in range(3) : 
            for j in range(3) : 
                v = ' '
                if self.board[i][j] == 1 : 
                    v = 'X' 
                elif self.board[i][j] == -1 : 
                    v = 'O'
                end = ''
                if j != 2 : 
                    end = ' | '
                print(v, end=end) 
            print()
            if i != 2 :
                print('---------')
        print() 
    
    def is_terminal(self) :  
        """
        returns 0 in case of draw, 
                1 in case the human player wins. 
                -1 in case the bot wins.
                -2 otherwise. 
        """
        def is_draw() :  
            if not np.any(self.board == 0) : 
                return True 
            return False 

        def is_valid(line) : 
            if len(np.unique(line)) == 1\
            and line[0] != 0 : 
                return True
            return False

        def check_diagonal() : 
            if is_valid(np.diag(self.board)) \
            or is_valid(np.diag(np.fliplr(self.board))) : 
                return True 
            else : 
                return False 

        def check_row_col() : 
            for i in range(3) : 
                if is_valid(self.board[:,i]) : 
                    return True, self.board[0,i] 
                if is_valid(self.board[i,:]) : 
                    return True, self.board[i][0] 
            return False, None 

        if check_diagonal() : 
            return self.board[1][1] 
        ok, v = check_row_col()
        if ok : 
            return v 
        if is_draw() : 
            return 0  
        return -2 

Next cell implements the minimax algorithm. The implementation strictly follows that of `AI: a modern approach`. Alpha-Beta pruning is used to shrink the search space. Since tic-tac-toe is 
an easy game, the problem is solved simply using minimax, Therefore the bot cannot be defeated and best you can get is a draw. Using minimax alone was enough to solve tic-tac-toe. However the algorithm still needed some time to explore the tree at first step. Using alpha-beta pruning reduced time to calculate the first move to `590ms` which is a huge improvement over previously `14.4s`.

In [8]:
from math import inf
def minimax(state, player) : 
    if player == human : 
        _, move = max_(state, -inf, inf) 
    else : 
        _, move = min_(state, -inf, inf)
    return move
    
def max_(state, alpha, beta) : 
    
    v = state.is_terminal() 
    if v != -2 :
        return v, None 
    v, move = -inf, None
     
    for action in state.next_actions() : 
        v_, _ = min_(state.next_state(*action, human), alpha, beta)
        if v_ > v : 
            v, move = v_, action 
            alpha = max(v, alpha)
        if v >= beta : 
            return v, move 
    return v, move

def min_(state, alpha, beta) : 
    v = state.is_terminal() 
    if v != -2 : 
        return v, None 
    v, move = inf, None
    for action in state.next_actions() : 
        v_, _ = max_(state.next_state(*action, bot), alpha, beta)
        if v_ < v : 
            v, move = v_, action 
            beta = min(beta, v) 
        if v <= alpha : 
            return v, move 
    return v, move 

Next I'll walk through steps involved in a game to test the bot. 

In [11]:
game = State() 
game.move(0, 0, human) 
x, y = minimax(game, bot) 
game.move(x, y, bot) 
game.print_board()

X |   |  
---------
  | O |  
---------
  |   |  



In [12]:
game.move(2, 2, human)
x, y = minimax(game, bot) 
game.move(x, y, bot) 
game.print_board()

X | O |  
---------
  | O |  
---------
  |   | X



In [13]:
game.move(2, 1, human)
x, y = minimax(game, bot) 
game.move(x, y, bot) 
game.print_board()

X | O |  
---------
  | O |  
---------
O | X | X



In [14]:
game.move(0, 2, human)
x, y = minimax(game, bot) 
game.move(x, y, bot) 
game.print_board()

X | O | X
---------
  | O | O
---------
O | X | X



It is evident that the bot was able to get a draw. What if we make a mistake? Is it smart enough to exploit our mistake and win the game?

In [25]:
game = State() 
game.move(0, 0, human)
x, y = minimax(game, bot) 
game.move(x, y, bot)
game.move(2, 2, human)
x, y = minimax(game, bot) 
game.move(x, y, bot)
game.print_board()

X | O |  
---------
  | O |  
---------
  |   | X



In [24]:
game.move(2, 0, human)
x, y = minimax(game, bot) 
game.move(x, y, bot)
game.print_board() 

X | O |  
---------
  | O |  
---------
X | O | X



Bot won ! There are however subtle ways that can make one think that the bot is not doing optimal. If for instance we move to (1, 0) instead of (2, 0) in our blunder move something unexpected would happen. 

In [26]:
game.move(1, 0, human)
x, y = minimax(game, bot) 
game.move(x, y, bot)
game.print_board() 

X | O |  
---------
X | O |  
---------
O |   | X



Bot could have won simply by moving to (2, 1). It chose (2, 0) ,however. The reason is obvious. The bot checks the next moves in a top-to-bottom, left-to-right manner. Thus it considers (2, 0) first. After checking (2, 0) it can deterministically tell that moving to (2, 0) can lead to a win. Therefore after checking (2, 1) it won't change it's move to (2, 1) since (2, 1) and (2, 0) are both winner moves returning `1`. In other words the algorithm is ignorant of the depth leading to a win. This makes no difference and we can easily make minimax to prefer choices with less depth. 

You can play tic-tac-toe and try everything for yourself in the cell below. 

In [213]:
from IPython.display import clear_output 
game = State() 
def is_end_game() : 
    t = game.is_terminal() 
    return True if t != -2 else False 

while True : 
    if is_end_game() : 
        break 
    game.print_board() 
    print("choose your move: ") 
    x, y = map(int ,input().split(','))
    game.move(x, y, human)
    clear_output() 
    if is_end_game() : 
        break 
    x_, y_ = minimax(game, bot) 
    game.move(x_, y_, bot)
p
game.print_board()
t = game.is_terminal() 
if t == 1: 
    print("You won!") 
elif t == -1 : 
    print("Bot won!") 
elif t == 0 : 
    print("draw!") 

X | O | X
---------
O | O | X
---------
X | X | O

draw!
